In [1]:
# Install required libraries
%pip install datasets transformers huggingface_hub -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 1.1 MB/s eta 0:00:00


In [2]:
# Login to HF hub
notebook_login()
     

NameError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.4 MB/s eta 0:00:00


In [4]:
# Import key libraries
import numpy as np
import os
import pandas as pd
import re

from datasets import load_dataset
import evaluate
from huggingface_hub import notebook_login
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TrainingArguments, Trainer
from transformers import AutoTokenizer, RobertaModel
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter('ignore')

In [5]:
# Disable Weights & Biases
os.environ["WANDB_DISABLED"] = "true"

In [6]:
# Loading the datasets
train_df = pd.read_csv('/content/drive/MyDrive/Train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Test.csv')

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5177 entries, 0 to 5176
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   5177 non-null   object
 1   safe_text  5176 non-null   object
dtypes: object(2)
memory usage: 81.0+ KB


## Lets check null values

In [9]:
train_df.isna().sum()
     

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

In [10]:
train_df[train_df.isna().any(axis=1)]

,tweet_id,safe_text,label,agreement
4798,RQMQ0L2A,#lawandorderSVU,NaN,NaN
4799,I cannot believe in this day and age some pare...,1,0.666667,NaN


In [11]:
test_df[test_df.isna().any(axis=1)]

,tweet_id,safe_text
2024,Dr. JAMES SHANNON,NaN


In [12]:
# Drop the rows with nulls from the training data
train_df.dropna(inplace = True)
     

In [13]:
#Rechecking the data types
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   9999 non-null   object 
 1   safe_text  9999 non-null   object 
 2   label      9999 non-null   float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 390.6+ KB


In [14]:
# Reconvert it to  respective data types
train_df['label'] = train_df['label'].astype(float)


# Training

Spliting train data into training and validation sets

In [15]:
train, eval = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'],shuffle=True)

# Fine-tuning the Roberta-large Model










In [16]:
# Instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-large", num_labels=3)

In [17]:
# Save split data subsets
train.to_csv('/content/drive/MyDrive/Train_subset.csv', index=False)
eval.to_csv('/content/drive/MyDrive/Eval_subset.csv', index=False)
     

In [18]:
# Load the subsetted data
data = load_dataset ("csv", 
                    data_files={"train": '/content/drive/MyDrive/Train_subset.csv',
                                "eval": '/content/drive/MyDrive/Eval_subset.csv'}, 
                    encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-04879f568e798e35/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# Define helper functions
## Function to transform labels
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}

## Function to tokenize data
def tokenize_data(example):
    return tokenizer(example['safe_text'], padding='max_length')

In [20]:
# Tokenize the tweets
dataset = data.map(tokenize_data, batched=True)

# Transform	labels and limit the columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [30]:
# Define training arguments
training_args = TrainingArguments(
    "covid_tweets_sentiment_analysis_model", 
    num_train_epochs=10,
    load_best_model_at_end=True,
    evaluation_strategy="epoch",
    save_strategy="epoch"
    )

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
# Load the pretrained model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("roberta-large", num_labels=3)
     

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.weight', 'clas

In [23]:
# Define evaluation metrics
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"rmse": mean_squared_error(labels, predictions, squared=False)}

In [24]:
# Formed  train and evaluation datasets
train_dataset = dataset["train"].shuffle(seed=15) 
eval_dataset = dataset["eval"].shuffle(seed=15)

In [25]:
#converting training data to PyTorch tensors 
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [26]:
# building the trainer
trainer = Trainer(model=model, args=training_args, 
                  train_dataset=train_dataset, 
                  eval_dataset= eval_dataset,compute_metrics=compute_metrics)
trainer.train()

OutOfMemoryError: ignored

In [ ]:
# reinstating the trainer for evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Launch the final evaluation 
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.5777072906494141,
 'eval_rmse': 0.6410928169929843,
 'eval_runtime': 31.1476,
 'eval_samples_per_second': 64.21,
 'eval_steps_per_second': 8.026}

In [ ]:
# Push model and tokenizer to HugginFace 
model.push_to_hub("Kwasiasomani/covid-tweet-sentiment-analyzer-distilbert")
tokenizer.push_to_hub("Kwasiasomani/covid-tweet-sentiment-analyzer-distilbert")

CommitInfo(commit_url='https://huggingface.co/Kwasiasomani/covid-tweet-sentiment-analyzer-distilbert/commit/97ef4f325827855538622fdd67b02ff6615cfc9a', commit_message='Upload tokenizer', commit_description='', oid='97ef4f325827855538622fdd67b02ff6615cfc9a', pr_url=None, pr_revision=None, pr_num=None)